In [1]:
import numpy as np
import pandas as pd

## Linsentypen

- A Sammellinse
- B Sammellinse
- G Sammellinse
- E Streulinse

## Allgemeine Settings

In [2]:
pos_lampe = 0e-2
u_pos_lampe = 0.1e-2

pos_linse_a = 5e-2
u_pos_linse_a = 0.1e-2

pos_gitter = 20e-2
u_pos_gitter = 0.1e-2

## Autokollimationsmethode Einfachlinsen

In [7]:
d_linse_spiegel = 3e-2
u_d_linse_spiegel = 0.1e-2

pd.read_excel("OPA.xlsx", sheet_name="AutokollimationEinfachlinsen")

,G,B
0,27.5,30.2
1,27.4,30.3
2,27.4,30.1
3,27.6,30.1
4,27.6,30.2


## Besselmethode Einfachlinsen

In [9]:
gittergroesse = 5e-3
u_gittergroesse = 0.1e-2

gittergroesse_schirm = 20e-3
u_gittergroesse_schirm = 0.1e-3

g_pos_schirm = 69.5e-2
u_g_pos_schirm = 0.1e-2

e_pos_schirm = 84.1e-2
u_e_pos_schirm = 0.1e-2

pd.read_excel("OPA.xlsx", sheet_name="BesselEinfachlinsen")

,G gross,G klein,E gross,E klein
0,29.4,61.1,32.9,72.0
1,29.4,60.9,32.5,71.9
2,29.1,60.8,32.9,72.0
3,29.2,60.6,32.7,72.2
4,29.3,60.8,32.5,72.0


## Autokollimationsmethode Linsensysteme

In [10]:
d_linse_spiegel = 3e-2
u_d_linse_spiegel = 0.1e-2

pd.read_excel("OPA.xlsx", sheet_name="AutokollimationLinsensystem")

,G-E-Spiegel,E-G-Spiegel
0,36.6,30.8
1,36.5,30.6
2,36.7,30.8
3,36.6,30.5
4,36.7,30.6


## Besselmethode Linsensysteme

In [11]:
g_e_pos_schirm = 107.5e-2
u_g_e_pos_schirm = 0.1e-2

e_g_pos_schirm = 104.6e-2
u_e_g_pos_schirm = 0.1e-2

pd.read_excel("OPA.xlsx", sheet_name="BesselLinsensystem")

,G-E-gross,G-E-klein,E-G-gross,E-G-klein
0,39.2,94.1,33.9,84.7
1,39.1,94.4,34.2,84.8
2,39.3,94.0,34.2,85.1
3,39.4,94.0,33.7,85.1
4,39.4,94.4,34.1,85.0


## Abbeverfahren Linsensysteme

In [12]:
pos_schirm = 150e-2
u_pos_schirm = 0.1e-2

pd.read_excel("OPA.xlsx", sheet_name="AbbeE-G-Schirm")

,Schirm,Vergroesserung,Position G,Verkleinerung,Position G.1
0,150,34.40,33.0,?,132.1
1,125,24.55,32.7,1,106.1
2,100,17.60,33.4,1.35,79.9
3,75,7.65,38.0,3.2,52.1


In [13]:
pd.read_excel("OPA.xlsx", sheet_name="AbbeG-E-Schirm")

,Schirm,Vergroesserung,Position G,Verkleinerung,Position G.1
0,150,33.75,39.5,?,137.7
1,125,25.45,38.5,1.5,112.1
2,100,17.45,40.2,1.5,85.6
3,75,8.15,44.8,3.3,56.3
